In [ ]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import os
import re
# 调库

In [ ]:

df_test = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/test.csv")
df_sub = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/sample_submission.csv")
df_stores = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/stores.csv")
df_items = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/items.csv")
df_trans = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/transactions.csv")
df_oil = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/oil.csv")
df_holiday = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/holidays_events.csv")

df_train = pd.read_csv("../input/after2017/after2017.csv")
df_train_test = pd.read_csv("../input/after2016/after2016.csv")

In [ ]:
df_train

In [ ]:
df_train['store_nbr'].unique()

In [ ]:
df_test

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])

In [ ]:
sns.displot(df_train['unit_sales'].sample(1000))

In [ ]:
sns.displot(np.log1p(df_train['unit_sales'].sample(1000)))

In [ ]:
df_test

In [ ]:
df_trans

In [ ]:
df_items

In [ ]:
df_holiday

In [ ]:
df_train['date'].min(), df_train['date'].max()

In [ ]:
df_test['date'].min(), df_test['date'].max()

In [ ]:
df_train['item_nbr'].unique().shape[0]#4018

In [ ]:
df_train['unit_sales'].describe()

In [ ]:
sns.displot(df_train['unit_sales'].sample(1000))

In [ ]:
# 开始建立模型
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error#mse
import lightgbm as lgb

In [ ]:
df_train = pd.read_csv('../input/after2017/after2017.csv',
usecols=[2, 3, 4, 5, 6],#这个表示第0列不读取，直接跳过，
dtype={'onpromotion': bool},#dtype就是将onpromotion哪一行直接转化为bool值
converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
parse_dates=["date"])
# df_train = pd.read_csv("../input/after2017/after2017.csv")
# 读train

In [ ]:
# df_train

In [ ]:

df_test = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/test.csv",
usecols=[0, 1, 2, 3, 4],
dtype={'onpromotion': bool},
parse_dates=["date"]).set_index(['store_nbr', 'item_nbr', 'date'])
# 读test并处理

In [ ]:
# df_test

In [ ]:
items = pd.read_csv("../input/favoritagrocerysalesforecastingextracted/items.csv").set_index("item_nbr")
# 读items

In [ ]:
# items

In [ ]:
# df_test[["onpromotion"]].unstack(level=-1).fillna(False)

In [ ]:
promo_2017_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [ ]:
# promo_2017_train

In [ ]:
# promo_2017_test

In [ ]:
# df_train[["onpromotion"]].unstack(level=-1).fillna(False)

In [ ]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values
    })
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X